# Final Data Upload File

In [2]:
import pandas as pd
import re
import requests

In [3]:
muxdf = pd.read_csv('mux.tsv', na_values='inf', sep='\t')
muxdf.head()

,Substrate,control_mean,CHIR99021_mean,CHIR99021_fold_change,CHIR99021_p-value,CHIR99021_ctrlCV,CHIR99021_treatCV,GSK2334470_mean,GSK2334470_fold_change,GSK2334470_p-value,...,AZD5363_mean,AZD5363_fold_change,AZD5363_p-value,AZD5363_ctrlCV,AZD5363_treatCV,BX912_mean,BX912_fold_change,BX912_p-value,BX912_ctrlCV,BX912_treatCV
0,ZZZ3(S82),3.837478e+08,3.350998e+08,0.873229,0.490620,0.248829,0.190148,5.545046e+08,1.444971,0.060763,...,4.508884e+08,1.174960,0.338517,0.248829,0.129146,3.691267e+08,0.961899,0.846261,0.248829,0.218934
1,ZZEF1(S1464),4.324388e+08,2.466291e+08,0.570322,0.182476,0.361260,0.590201,2.764129e+08,0.639195,0.141881,...,1.174577e+08,0.271617,0.020323,0.361260,0.658044,6.894412e+08,1.594310,0.078707,0.361260,0.204332
2,ZYX(S344),1.734734e+10,1.377140e+10,0.793862,0.422038,0.329156,0.317195,2.639660e+10,1.521651,0.068107,...,1.668084e+10,0.961579,0.860644,0.329156,0.159568,1.273611e+10,0.734182,0.401933,0.329156,0.531620
3,ZYX(S308),2.504079e+09,1.527156e+09,0.609867,0.116059,0.247867,0.446213,1.534257e+09,0.612703,0.042220,...,1.331806e+09,0.531855,0.058811,0.247867,0.460131,2.143351e+09,0.855944,0.563849,0.247867,0.379531
4,ZYX(S281),4.483330e+09,4.509845e+09,1.005914,0.968388,0.196722,0.150060,3.411014e+09,0.760821,0.205854,...,5.468233e+09,1.219681,0.230357,0.196722,0.169143,3.420002e+09,0.762826,0.088660,0.196722,0.062186


In [ ]:
#mux.tsv has data for 5 inhibitors
#shares data for substrate and control mean

In [4]:
muxdf=muxdf.dropna(axis='index', how='any') #removes all the NaN rows 
muxdf=muxdf[~muxdf.Substrate.str.contains("None")]
M= r"\([M]\d+\)" #matches M in brackets with one or more digits
muxdf=muxdf[~muxdf.Substrate.str.contains(M)] #drops rows with M residue
phos=muxdf.Substrate.str.findall(r"\((.\d+)").apply(','.join, 1)
muxdf.insert(1, "Phosphosite", phos, True) #inserts phosphosite data as the second column
muxdf[["Substrate"]]=muxdf.Substrate.str.extract(r"(.+)\(")
muxdf.head()

,Substrate,Phosphosite,control_mean,CHIR99021_mean,CHIR99021_fold_change,CHIR99021_p-value,CHIR99021_ctrlCV,CHIR99021_treatCV,GSK2334470_mean,GSK2334470_fold_change,...,AZD5363_mean,AZD5363_fold_change,AZD5363_p-value,AZD5363_ctrlCV,AZD5363_treatCV,BX912_mean,BX912_fold_change,BX912_p-value,BX912_ctrlCV,BX912_treatCV
0,ZZZ3,S82,3.837478e+08,3.350998e+08,0.873229,0.490620,0.248829,0.190148,5.545046e+08,1.444971,...,4.508884e+08,1.174960,0.338517,0.248829,0.129146,3.691267e+08,0.961899,0.846261,0.248829,0.218934
1,ZZEF1,S1464,4.324388e+08,2.466291e+08,0.570322,0.182476,0.361260,0.590201,2.764129e+08,0.639195,...,1.174577e+08,0.271617,0.020323,0.361260,0.658044,6.894412e+08,1.594310,0.078707,0.361260,0.204332
2,ZYX,S344,1.734734e+10,1.377140e+10,0.793862,0.422038,0.329156,0.317195,2.639660e+10,1.521651,...,1.668084e+10,0.961579,0.860644,0.329156,0.159568,1.273611e+10,0.734182,0.401933,0.329156,0.531620
3,ZYX,S308,2.504079e+09,1.527156e+09,0.609867,0.116059,0.247867,0.446213,1.534257e+09,0.612703,...,1.331806e+09,0.531855,0.058811,0.247867,0.460131,2.143351e+09,0.855944,0.563849,0.247867,0.379531
4,ZYX,S281,4.483330e+09,4.509845e+09,1.005914,0.968388,0.196722,0.150060,3.411014e+09,0.760821,...,5.468233e+09,1.219681,0.230357,0.196722,0.169143,3.420002e+09,0.762826,0.088660,0.196722,0.062186


In [5]:
def find_sub_gene(entry):
    if re.match(r".+_HUMAN", entry):
        URL = 'http://www.uniprot.org/uniprot/?query==mnemonic:'+entry+'&columns=genes(PREFERRED)&format=tab'
        r = requests.get(URL)
        content = r.text.splitlines()
        gene_name=content[1:2]        
        return str(gene_name)  #returns gene as a string 
    else:
        return entry           #if entry doesn't match regex, return the entry (gene name)

In [7]:
muxdf.Substrate=muxdf.apply(lambda row: find_sub_gene(row["Substrate"]), axis=1)
muxdf.Substrate=muxdf.Substrate.str.strip("[]").str.strip("''") #remove [] and ''
muxdf.Substrate.replace("", np.nan, inplace=True) #replace blank spaces with NaN
muxdf.dropna(subset=["Substrate"], inplace=True)

#muxdf.to_csv("mux.csv")
muxdf.head()

NameError: name 'np' is not defined